In [1]:
class Simulation:
    def __init__(self, parameters):
        self.parameters = parameters
        self.population_species_history_1 = [parameters.initial_population_species_1]
        self.population_species_history_2 = [parameters.initial_population_species_2]
        self.z_m1_history = [parameters.z_m1_initial]
        self.z_m2_history = [parameters.z_m2_initial]
        self.z_m1 = parameters.z_m1_initial
        self.z_m2 = parameters.z_m2_initial

    def next_generation_ecology(self):
        N1 = self.population_species_history_1[-1]
        N2 = self.population_species_history_2[-1]
      
    
        if N1 > 0.01:
            next_N1 = (N1 / (N1 + self.parameters.strength_of_interference* (1 - self.z_m2) * N2)) * (self.parameters.b1 * N1 / (1 + self.parameters.alpha11 * N1 + self.parameters.alpha12 * N2))
            next_N1 = max(next_N1, 0)
        else:
            next_N1 = 0

        if N2 > 0.01:
            next_N2 = (N2 / (N2 + self.parameters.strength_of_interference * (1 - self.z_m1) * N1)) * (self.parameters.b2 * N2 / (1 + self.parameters.alpha22 * N2 + self.parameters.alpha21 * N1))
            next_N2 = max(next_N2, 0)
        else:
            next_N2 = 0

        self.population_species_history_1.append(next_N1)
        self.population_species_history_2.append(next_N2)

    def next_generation_traits(self, N1, N2):
        if N1 > 0.01:
            W_m1 = 1 - self.parameters.cost_to_males*(((1 - self.z_m1) * (N2 / (N2 + N1))) ** (1/self.parameters.cost_to_males))
            dW_m1_dz_m1 =  ((1 - self.z_m1)** (1/self.parameters.cost_to_males - 1)) * ((N2 / (N2 + N1)) ** (1/self.parameters.cost_to_males))
            delta_z_m1 = self.parameters.V_A1 * (1 / W_m1) * dW_m1_dz_m1
            new_z_m1 = self.z_m1 + delta_z_m1
            
            if new_z_m1 <= 1:
                self.z_m1 = new_z_m1
            else:
                self.z_m1 = 1

            self.z_m1_history.append(self.z_m1)

        if N2 > 0.01:
            W_m2 = 1 - self.parameters.cost_to_males*(((1 - self.z_m2) * (N1 / (N1 + N2))) ** (1/self.parameters.cost_to_males))
            dW_m2_dz_m2 = ((1 - self.z_m2) ** (1/self.parameters.cost_to_males - 1)) * (N1 / (N1 + N2)) ** ((1/self.parameters.cost_to_males))
            delta_z_m2 = self.parameters.V_A2 * (1 / W_m2) * dW_m2_dz_m2
            new_z_m2 = self.z_m2 + delta_z_m2
            
            if new_z_m2 <= 1:
                self.z_m2 = new_z_m2
            else:
                self.z_m2 = 1

            self.z_m2_history.append(self.z_m2)
            
    def next_generation(self):
        self.next_generation_ecology()
        N1 = self.population_species_history_1[-1]
        N2 = self.population_species_history_2[-1]
        self.next_generation_traits(N1, N2)
        return N1, N2

    def run_simulation(self):
        for t in range(self.parameters.num_generations - 1):
            N1, N2 = self.next_generation()
            if N1 < 0.01 or N2 < 0.01:
                break

        return self.population_species_history_1, self.population_species_history_2, self.z_m1_history, self.z_m2_history

class outcome_stats_and_plots:
    def __init__(self, population_species_1, population_species_2, z_m1_history, z_m2_history, parameters):
        self.population_species_1 = population_species_1
        self.population_species_2 = population_species_2
        self.z_m1_history = z_m1_history
        self.z_m2_history = z_m2_history
        self.parameters = parameters

        self.outcome = None
        self.number_of_generations_until_extinctioin = None
        self.final_population_species_1 = None
        self.final_population_species_2 = None
        self.final_z_m1 = None
        self.final_z_m2 = None

        self.outcome_stats()

    def plot_population_dynamics_and_trait_evolution(self):
        self.plot_population_dynamics()
        self.plot_traits_evolution()

    def plot_population_dynamics(self):
        fig = go.Figure()

        # Population Species 1
        fig.add_trace(go.Scatter(x=list(range(len(self.population_species_1))), 
                                 y=self.population_species_1, 
                                 mode='lines', 
                                 name='Slow-evolving Species', 
                                 line=dict(color='blue')))

        # Population Species 2
        fig.add_trace(go.Scatter(x=list(range(len(self.population_species_2))), 
                                 y=self.population_species_2, 
                                 mode='lines', 
                                 name='Fast-evolving Species', 
                                 line=dict(color='red')))

        fig.update_layout(
            # title='Population Dynamics of Two Competing Species',
            xaxis_title='Generation',
            yaxis_title='Population Numbers',
            legend_title='Species',
            # template='plotly_dark'  # Set the dark theme
        )

        pio.show(fig)  # Use pyo.plot for script execution, pio.show for Jupyter Notebooks

    def plot_traits_evolution(self):
        fig = go.Figure()

        # Mean Trait z_m1
        fig.add_trace(go.Scatter(x=list(range(len(self.z_m1_history))), 
                                 y=self.z_m1_history, 
                                 mode='lines', 
                                 name='Mean Trait (slow evolver)', 
                                 line=dict(color='blue')))

        # Mean Trait z_m2
        fig.add_trace(go.Scatter(x=list(range(len(self.z_m2_history))), 
                                 y=self.z_m2_history, 
                                 mode='lines', 
                                 name='Mean Trait (Fast Evolver)', 
                                 line=dict(color='red')))

        fig.update_layout(
            xaxis_title='Generation',
            yaxis_title='Mean Trait Value (Discrimiation)',
            legend_title='Traits',
            # template='plotly_dark'  # Set the dark theme
        )

        pio.show(fig)  # Use pyo.plot for script execution, pio.show for Jupyter Notebooks

    def outcome_stats(self):

        if self.population_species_1[-1] < 0.01 and self.population_species_2[-1] < 0.01:
            self.outcome = "both extinct"
        elif self.population_species_1[-1] < 0.01 and self.population_species_2[-1] > 0.01:
            self.outcome = "sp 2 (high-genetic-var) wins, sp 1 (low-genetic-var) extinct"
        elif self.population_species_2[-1] < 0.01 and self.population_species_1[-1] > 0.01:
            self.outcome = "sp 1 (low-genetic-var) wins, sp 2 (high-genetic-var) extinct"
        else:
            self.outcome = "both coexist"

        if len(self.population_species_1) < self.parameters.num_generations:
            self.number_of_generations_until_extinctioin = len(self.population_species_1)
            self.final_population_species_1 = self.population_species_1[-1]
            self.final_population_species_2 = self.population_species_2[-1]
            self.final_z_m1 = self.z_m1_history[-1]
            self.final_z_m2 = self.z_m2_history[-1]

    def print_outcome_stats(self):

        print("Outcome of simulation: ", self.outcome)    
        print("Number of generations until extinction: ", self.number_of_generations_until_extinctioin)
        print("Final population of species 1: ", self.final_population_species_1)
        print("Final population of species 2: ", self.final_population_species_2)
        print("Final trait value z_m1: ", self.final_z_m1)
        print("Final trait value z_m2: ", self.final_z_m2)

def run_simulation_and_plot_pop_dynamics_and_trait_evolution(outcome_stats_and_plots_instance):
    outcome_stats_and_plots_instance.plot_population_dynamics_and_trait_evolution()
    outcome_stats_and_plots_instance.print_outcome_stats()

def run_simulation_and_get_outcome_instance(parameters):
    simulation = Simulation(parameters)
    population_species_1, population_species_2, z_m1_history, z_m2_history = simulation.run_simulation()
    outcome_stats_and_plots_instance = outcome_stats_and_plots(population_species_1, population_species_2, z_m1_history, z_m2_history, parameters)
    
    return outcome_stats_and_plots_instance
